In [10]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

from plotly import plotly
import plotly.offline as offline
import plotly.graph_objs as go
offline.init_notebook_mode()
from collections import Counter

In [11]:
project_data = pd.read_csv('F:/aai/code/Assignments_DonorsChoose_2018/train_data.csv').sample(n=60000)
resource_data = pd.read_csv('F:/aai/code/Assignments_DonorsChoose_2018/resources.csv')

In [12]:
print("Number of data points in train data", project_data.shape)
print('-'*50)
print("The attributes of data :", project_data.columns.values)

Number of data points in train data (60000, 17)
--------------------------------------------------
The attributes of data : ['Unnamed: 0' 'id' 'teacher_id' 'teacher_prefix' 'school_state'
 'project_submitted_datetime' 'project_grade_category'
 'project_subject_categories' 'project_subject_subcategories'
 'project_title' 'project_essay_1' 'project_essay_2' 'project_essay_3'
 'project_essay_4' 'project_resource_summary'
 'teacher_number_of_previously_posted_projects' 'project_is_approved']


In [13]:
print("Number of data points in train data", resource_data.shape)
print(resource_data.columns.values)
resource_data.head(2)

Number of data points in train data (1541272, 4)
['id' 'description' 'quantity' 'price']


id                                        description  quantity  \
0  p233245  LC652 - Lakeshore Double-Space Mobile Drying Rack         1   
1  p069063        Bouncy Bands for Desks (Blue support pipes)         3   

    price  
0  149.00  
1   14.95

In [14]:
catogories = list(project_data['project_subject_categories'].values)
# remove special characters from list of strings python: https://stackoverflow.com/a/47301924/4084039

# https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
# https://stackoverflow.com/questions/23669024/how-to-strip-a-specific-word-from-a-string
# https://stackoverflow.com/questions/8270092/remove-all-whitespace-in-a-string-in-python
cat_list = []
for i in catogories:
    temp = ""
    # consider we have text like this "Math & Science, Warmth, Care & Hunger"
    for j in i.split(','): # it will split it in three parts ["Math & Science", "Warmth", "Care & Hunger"]
        if 'The' in j.split(): # this will split each of the catogory based on space "Math & Science"=> "Math","&", "Science"
            j=j.replace('The','') # if we have the words "The" we are going to replace it with ''(i.e removing 'The')
        j = j.replace(' ','') # we are placeing all the ' '(space) with ''(empty) ex:"Math & Science"=>"Math&Science"
        temp+=j.strip()+" " #" abc ".strip() will return "abc", remove the trailing spaces
        temp = temp.replace('&','_') # we are replacing the & value into 
    cat_list.append(temp.strip())
    
project_data['clean_categories'] = cat_list
project_data.drop(['project_subject_categories'], axis=1, inplace=True)


In [15]:
sub_catogories = list(project_data['project_subject_subcategories'].values)
# remove special characters from list of strings python: https://stackoverflow.com/a/47301924/4084039

# https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
# https://stackoverflow.com/questions/23669024/how-to-strip-a-specific-word-from-a-string
# https://stackoverflow.com/questions/8270092/remove-all-whitespace-in-a-string-in-python

sub_cat_list = []
for i in sub_catogories:
    temp = ""
    # consider we have text like this "Math & Science, Warmth, Care & Hunger"
    for j in i.split(','): # it will split it in three parts ["Math & Science", "Warmth", "Care & Hunger"]
        if 'The' in j.split(): # this will split each of the catogory based on space "Math & Science"=> "Math","&", "Science"
            j=j.replace('The','') # if we have the words "The" we are going to replace it with ''(i.e removing 'The')
        j = j.replace(' ','') # we are placeing all the ' '(space) with ''(empty) ex:"Math & Science"=>"Math&Science"
        temp +=j.strip()+" "#" abc ".strip() will return "abc", remove the trailing spaces
        temp = temp.replace('&','_')
    sub_cat_list.append(temp.strip())

project_data['clean_subcategories'] = sub_cat_list
project_data.drop(['project_subject_subcategories'], axis=1, inplace=True)

In [16]:
# merge two column text dataframe: 
project_data["essay"] = project_data["project_essay_1"].map(str) +\
                        project_data["project_essay_2"].map(str) + \
                        project_data["project_essay_3"].map(str) + \
                        project_data["project_essay_4"].map(str)

In [17]:
project_data.head(2)

Unnamed: 0       id                        teacher_id teacher_prefix  \
96294       43715  p009933  cbb440d8a6de4a631eae3fa1b1973c1d            Mr.   
1462       115870  p070729  a5480f0841d176b44cb7abe098ee777a           Mrs.   

      school_state project_submitted_datetime project_grade_category  \
96294           CA        2016-11-03 19:30:49            Grades 9-12   
1462            MI        2017-01-26 09:44:19            Grades 9-12   

                               project_title  \
96294         Enlarging the Smallest Details   
1462   Makerspace Magic Lands In The Library   

                                         project_essay_1  \
96294  My students are the children of migrant worker...   
1462   A majority of my students read significantly b...   

                                         project_essay_2 project_essay_3  \
96294  In my Intro to Ag Mechanics classes, I teach a...             NaN   
1462   The art supplies requested, such as clay, cons...             NaN   

      project_essay_4                           project_resource_summary  \
96294             NaN  My students need to see many of the finer deta...   
1462              NaN  My students need art supplies, such as clay, e...   

       teacher_number_of_previously_posted_projects  project_is_approved  \
96294                                             3                    1   
1462                                              2                    1   

                   clean_categories             clean_subcategories  \
96294  AppliedLearning Math_Science  College_CareerPrep Mathematics   
1462                     Music_Arts                      VisualArts   

                                                   essay  
96294  My students are the children of migrant worker...  
1462   A majority of my students read significantly b...

In [18]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [19]:
sent = decontracted(project_data['essay'].values[2000])
print(sent)
print("="*50)

I am a kindergarten teacher with big ideas to bring movement, play, and the arts back into the classroom. I believe in teaching even the youngest students to see themselves as learners who have different needs, and to give students tools to help them meet those needs. We are unique and wonderful. \r\n\r\nMy students and I are part of a wonderful adventure. We are part of a team of teachers, students, and families opening a brand new elementary school. My kids will be the first to attend our school from kindergarten through fifth grade. We work every day to build a strong community of learners and friends. We strive to turn a school that started empty into a home away from home...a place we are proud to come to every day to play, learn, and grow.You can never have enough construction paper in kindergarten. We use it all the time! This year, it is especially important to have a larger quantity of this important resource. My kids are coming into a brand new building, and art is going to b

In [20]:
# \r \n \t remove from string python: http://texthandler.com/info/remove-line-breaks-python/
sent = sent.replace('\\r', ' ')
sent = sent.replace('\\"', ' ')
sent = sent.replace('\\n', ' ')
print(sent)

I am a kindergarten teacher with big ideas to bring movement, play, and the arts back into the classroom. I believe in teaching even the youngest students to see themselves as learners who have different needs, and to give students tools to help them meet those needs. We are unique and wonderful.     My students and I are part of a wonderful adventure. We are part of a team of teachers, students, and families opening a brand new elementary school. My kids will be the first to attend our school from kindergarten through fifth grade. We work every day to build a strong community of learners and friends. We strive to turn a school that started empty into a home away from home...a place we are proud to come to every day to play, learn, and grow.You can never have enough construction paper in kindergarten. We use it all the time! This year, it is especially important to have a larger quantity of this important resource. My kids are coming into a brand new building, and art is going to be ou

In [21]:
#remove spacial character: https://stackoverflow.com/a/5843547/4084039
sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
print(sent)

I am a kindergarten teacher with big ideas to bring movement play and the arts back into the classroom I believe in teaching even the youngest students to see themselves as learners who have different needs and to give students tools to help them meet those needs We are unique and wonderful My students and I are part of a wonderful adventure We are part of a team of teachers students and families opening a brand new elementary school My kids will be the first to attend our school from kindergarten through fifth grade We work every day to build a strong community of learners and friends We strive to turn a school that started empty into a home away from home a place we are proud to come to every day to play learn and grow You can never have enough construction paper in kindergarten We use it all the time This year it is especially important to have a larger quantity of this important resource My kids are coming into a brand new building and art is going to be our avenue for creating own

In [22]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [23]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_essays = []
# tqdm is for printing the status bar
for sentance in tqdm(project_data['essay'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_essays.append(sent.lower().strip())

100%|███████████████████████████████████| 60000/60000 [01:08<00:00, 880.53it/s]


In [24]:
# after preprocesing
preprocessed_essays[2000]
project_data['essay'] = preprocessed_essays
project_data.drop(['project_essay_1','project_essay_2','project_essay_3','project_essay_4'], axis=1, inplace=True)
project_data.keys()

Index(['Unnamed: 0', 'id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category', 'project_title',
       'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'clean_categories', 'clean_subcategories', 'essay'],
      dtype='object')

In [25]:
def preProcessData(columnName):
    preprocessed_data = []
    columnData = project_data[columnName]
    for i in columnData:
        temp = ""
        # consider we have text like this "Math & Science, Warmth, Care & Hunger"
        if i is None:
            i = 'Othr'
        for j in i.split(','): # it will split it in three parts ["Math & Science", "Warmth", "Care & Hunger"]
            if 'The' in j.split(): # this will split each of the catogory based on space "Math & Science"=> "Math","&", "Science"
                j=j.replace('The','') # if we have the words "The" we are going to replace it with ''(i.e removing 'The')
            j = j.replace(' ','') # we are placeing all the ' '(space) with ''(empty) ex:"Math & Science"=>"Math&Science"
            temp +=j.strip()+" "#" abc ".strip() will return "abc", remove the trailing spaces
            temp = temp.replace('&','_')
            temp = temp.replace("'","")
            temp = temp.replace('.','')
        preprocessed_data.append(temp.strip())
        
    return preprocessed_data

In [26]:
project_data['clean_titles'] = preProcessData('project_title')
project_data.drop(['project_title'], axis=1, inplace=True)
print(project_data['clean_titles'][1:10])

1462                     MakerspaceMagicLandsInLibrary
86556                          TimetoClockINtoFinance!
102852            Ut-zah=Success\r\nLiteracyTechnology
14300                                       ABCsofLife
3238                         DevelopmentofYoungWriters
63738     EveryStudentShouldGettheChancetoMakePottery!
96840         FosteringFutureInnovatorsintheMakerWorld
50070                       TonerTonerToner WeNeedYou!
46870                             RecycleTunesinMyHead
Name: clean_titles, dtype: object


In [27]:
project_data['clean_proj_grade_cats'] = preProcessData('project_grade_category')
project_data.drop(['project_grade_category'], axis=1, inplace=True)
print(project_data['clean_proj_grade_cats'][1:10])

1462        Grades9-12
86556       Grades9-12
102852       Grades6-8
14300        Grades3-5
3238         Grades6-8
63738        Grades3-5
96840        Grades3-5
50070        Grades6-8
46870     GradesPreK-2
Name: clean_proj_grade_cats, dtype: object


In [28]:
project_data['clean_states'] = preProcessData('school_state')
project_data.drop(['school_state'], axis=1, inplace=True)
print(project_data['clean_states'][1:10])

1462      MI
86556     NC
102852    NM
14300     NY
3238      IL
63738     FL
96840     NC
50070     NY
46870     IL
Name: clean_states, dtype: object


In [29]:
project_data.teacher_prefix.replace(np.nan,'Othr',inplace=True)
project_data['clean_prefixes'] = preProcessData('teacher_prefix')
project_data.drop(['teacher_prefix'], axis=1, inplace=True)
print(project_data['clean_prefixes'][1:10])

1462      Mrs
86556     Mrs
102852    Mrs
14300      Mr
3238       Ms
63738     Mrs
96840     Mrs
50070      Ms
46870      Ms
Name: clean_prefixes, dtype: object


In [30]:
price_data = resource_data.groupby('id').agg({'price':'sum', 'quantity':'sum'}).reset_index()
project_data = pd.merge(project_data, price_data, on='id', how='left')

In [31]:
project_data = project_data.fillna(0)
resource_data = resource_data.fillna(0)

In [37]:
def makeOneHotEncoding(vocabulary, inputData):
    vectorizer = CountVectorizer()
    vectorizer.fit(vocabulary)

    _one_hot = vectorizer.transform(inputData)
    return _one_hot

In [33]:
from pandas import Series
from sklearn.preprocessing import MinMaxScaler

def normalizeData(trainData, testData):
    # train the normalization
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(trainData)
    # normalize the dataset and print the first 5 rows
    trainDataN = scaler.transform(trainData)
    testDataN = scaler.transform(testData)
    return trainDataN, testDataN

In [34]:
def batch_predict(clf, data):
    # roc_auc_score(y_true, y_score) the 2nd parameter should be probability estimates of the positive class
    # not the predicted outputs

    y_data_pred = []
    tr_loop = data.shape[0] - data.shape[0]%1000
    print(tr_loop)
    # consider you X_tr shape is 49041, then your cr_loop will be 49041 - 49041%1000 = 49000
    # in this for loop we will iterate unti the last 1000 multiplier
    for i in range(0, tr_loop, 1000):
        y_data_pred.extend(clf.predict_proba(data[i:i+1000])[:,1])
    # we will be predicting for the last data points
    if data.shape[0] > tr_loop:
        y_data_pred.extend(clf.predict_proba(data[tr_loop:])[:,1])
    
    return y_data_pred

In [35]:
y = project_data['project_is_approved'].values
X = project_data.drop(['project_is_approved'], axis=1)
X.head(1)

Unnamed: 0       id                        teacher_id  \
0       43715  p009933  cbb440d8a6de4a631eae3fa1b1973c1d   

  project_submitted_datetime  \
0        2016-11-03 19:30:49   

                            project_resource_summary  \
0  My students need to see many of the finer deta...   

   teacher_number_of_previously_posted_projects              clean_categories  \
0                                             3  AppliedLearning Math_Science   

              clean_subcategories  \
0  College_CareerPrep Mathematics   

                                               essay  \
0  my students children migrant workers better 90...   

                  clean_titles clean_proj_grade_cats clean_states  \
0  EnlargingtheSmallestDetails            Grades9-12           CA   

  clean_prefixes   price  quantity  
0             Mr  961.49         1

In [36]:
vectorizer = CountVectorizer(min_df=10,ngram_range=(1,4), max_features=2000)
vectorizer.fit(X['essay'].values) # fit has to happen only on train data

X_essay_bow = vectorizer.transform(X['essay'].values)

In [41]:
vectorizer = CountVectorizer()
vectorizer.fit(X['clean_titles'].values) # fit has to happen only on train data

X_title_bow = vectorizer.transform(X['clean_titles'].values)

In [42]:
X_state_ohe = makeOneHotEncoding(X['clean_states'].values, X['clean_states'].values)
X_prefix_ohe = makeOneHotEncoding(X['clean_prefixes'].values, X['clean_prefixes'].values)
X_pgc_ohe = makeOneHotEncoding(X['clean_proj_grade_cats'].values, X['clean_proj_grade_cats'].values)
X_catg_ohe = makeOneHotEncoding(X['clean_categories'].values,X['clean_categories'].values)
X_subcatg_ohe = makeOneHotEncoding(X['clean_subcategories'].values, X['clean_subcategories'].values)

In [43]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()

X_price_norm = normalizer.transform(X['price'].values.reshape(-1,1))
X_quantity_norm = normalizer.transform(X['quantity'].values.reshape(-1,1))

In [45]:
from scipy.sparse import hstack
X_ = hstack((X_catg_ohe, X_subcatg_ohe, X_state_ohe, X_prefix_ohe, X_pgc_ohe, 
               X_essay_bow,X_title_bow,X_price_norm, X_quantity_norm)).tocsr()

print("Final Data matrix")
print(X_.shape)
print("="*100)

Final Data matrix
(60000, 62477)


In [46]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_, project_data['project_is_approved'].values)

In [47]:
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data)
graph.render("dt_bow") 

'dt_bow.pdf'

In [48]:
vectorizer = TfidfVectorizer(min_df=10, max_features=2000)
vectorizer.fit(X['essay'].values) # fit has to happen only on train data

X_essay_bow = vectorizer.transform(X['essay'].values)

In [49]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X['clean_titles'].values) # fit has to happen only on train data

X_title_bow = vectorizer.transform(X['clean_titles'].values)

In [50]:
X_ = hstack((X_catg_ohe, X_subcatg_ohe, X_state_ohe, X_prefix_ohe, X_pgc_ohe, 
               X_essay_bow,X_title_bow,X_price_norm, X_quantity_norm)).tocsr()

print("Final Data matrix")
print(X_.shape)
print("="*100)

Final Data matrix
(60000, 62477)


In [51]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_, project_data['project_is_approved'].values)

In [52]:
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data)
graph.render("dt_tfidf") 

'dt_tfidf.pdf'